# SIMULATION

In this file the script for the simulation of the whole spin chain. 

THe initial condisions are created perturbing the thermal state of the whole chain with a random unitary operator acting only on the first 2 spins.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import quimb as qu
import quimb.tensor as qtn
import quimb.linalg.base_linalg as la
from itertools import product

In [3]:
### PROBLEM PARAMETERS
L = 7        # chain length
omega = 1     # Rabi frequency
V = 0.1       # B-S interaction strength
V_ = 1        # bath interaction strength
beta = 0.1    # inverse temperature for the bath

T = 10        # total time of the simulation
dt = 0.1      # time step for coherece vector time evolution

dims = [2]*L # overall space of L qbits

In [7]:
### BUILDING THE HAMILTONIAN

# fisrt I build the operators tensor identity on ancilla states
I = qu.pauli('I')
X = qu.pauli('X') & I
Z = qu.pauli('Z') & I
nn_op = (1 + Z)/2 & (1 + Z)/2

print(f'Building Hamiltonian for L={L} \n')
# the hamiltonian
H1 = {i: (omega/2)*X for i in range(L)}
H2 = {None: nn_op, (L-1, 0): V*nn_op, (0, 1): V*nn_op, (1, 2): V*nn_op}

H = qtn.LocalHam1D(L=L, H2=H2, H1=H1, cyclic=True)

### BUILDING THE SPIN CHAIN

print('Building the spin chain: \n')
B = np.array([1, 0, 0, 1]).T/np.sqrt(2)
arrays = [B for i in range(L)]

psi = qtn.MPS_product_state(arrays, cyclic=True)#, site_ind_id='s{}')

psi.show()

print('\ndone')

Building Hamiltonian for L=7 

Building the spin chain 

 1 1 1 1 1 1 1 1 
+─●─●─●─●─●─●─●─+
  │ │ │ │ │ │ │  

done


In [ ]:
### THE SIMULATION

# first I build the observables
observables_tebd = {}
results = {}
for ob1, ob2 in product(['I', 'X', 'Y', 'Z'], repeat=2):
    key = ob1 + '1' + ob2 + '2'
    results[key] = []
    observables_tebd[key] = []

# dropping the identity
observables_tebd.pop('I1I2')
results.pop('I1I2')
    
# create the object
tebd = qtn.TEBD(psi, H)

# cutoff for truncating after each infenitesimal-time operator application
tebd.split_opts['cutoff'] = 1e-10

t_ = np.linspace(0, T, 100)
keys = results.keys()

# I can either set a timestep or a tolerance and the timestep is calculated accordingly
# generate the state at each time in t (same as exact simulation)
#     and target error 1e-3 for whole evolution
for psit in tebd.at_times(t_, tol=1e-3):
    for key in keys:
        ob1 = key[0]
        ob2 = key[2]
        #results[key].append(qtn.expec_TN_1D(psit.H, observables_tebd[key], psit))
        results[key].append(psit.H @ psit.gate(qu.pauli(ob1)&qu.pauli(ob2), (0, 1)))

To simulate the mixed state I use the ancilla method which employ a duplicate of the system as ancilla Hilbert space. Since each site and it's counterpart in the ancilla space are maximally entangled I want to keep them nearby. For this reason I put the physical particles (the one subjected to the Hmiltonian) on even indexes  0,2,4,6,... and ancilla particles on odd indexes 1,3,5,...

In [33]:
### BUILDING THE HAMILTONIAN
# tebd do not require an MPO, but a specification of the local interacting parts of the Hamiltonian

# fisrt I build the operators
I = qu.pauli('I')
X = qu.pauli('X')
Z = qu.pauli('Z')
nn_op = (1 + Z)/2 & (1 + Z)/2

# single particle H
#H1 = {(2*i): (omega/2)*X for i in range(0, L)}
#H1[None] = I

# two particle Hamiltonian
#H2 = {(i, i+2): V_*nn_op for i in range(4, 2*(L-1), 2)}
#H2[(2*L-2, 0)] = V*nn_op
#H2[(0, 1)] = V*nn_op
#H2[(1, 2)] = V*nn_op
#H2[None] = I

H1 = {i: (omega/2)*X for i in range(0, L)}
H1[None] = I
H2 = {(i, i+1): nn_op for i in range(2, L-1)} 
H2[(L-1, 0)] = V*nn_op
H2[(0, 1)] = V*nn_op
H2[(1, 2)] = V*nn_op
H2[None] = I

H = qtn.LocalHam1D(L=2*L, H2=H2, H1=H1)#, cyclic=True)


### BUILDING THE SPIN CHAIN

# binary version of the initial state
# 0 : up
# 1 : down
binary = '01'
for i in range(L-2):
    if i%2==0:
        binary += '0'
    else:
        binary += '1'
binary = '0'*L
print(f'|{binary}>')
        
# turning the state into a MPS
psi = qtn.MPS_computational_state(binary, cyclic=True)
psi.show()  # prints ascii representation of state


ValueError: operands could not be broadcast together with shapes (2,2) (4,4) 